In [1]:
#import libraries
import cv2 as cv
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import os

#define lengths of images in pixels
W=1440
H=1080
mapping = {'Car': 0, 'Motorcycle': 1, 'Truck': 2, 'Bus':3, 'Van':4, 'Bicycle':5, 'Pedestrian':6}

#Inputs jsons in pandas dataframe and outputs a properly formated txt file 
def format_json(urll):
    file_name_ext = (os.path.basename(urll))
    file_name = (os.path.splitext(os.path.splitext(file_name_ext)[0])[0])
    
    df = pd.read_json(urll, orient='columns')
    annotated_data = json_normalize(df['annotated_data'])
    
    if 'data' in annotated_data.columns:
        final_frame = annotated_data.drop(labels = 'data', axis = 1)
    else:
        final_frame = annotated_data
            
    df2 = pd.DataFrame(columns=final_frame.columns)
    df2_index = 0
    for row in final_frame.iterrows():
        one_row = row[1]
        for list_value in row[1]["bounding_box_data"]:
            one_row["bounding_box_data"] = list_value
            df2.loc[df2_index] = one_row
            df2_index += 1
            
    df2[list(df2["bounding_box_data"].head(1).tolist()[0].keys())] = df2["bounding_box_data"].apply(lambda x: pd.Series([x[key] for key in x.keys()]))
    
    df2['x']=df2['x']/W
    df2['y']=df2['y']/H
    df2['width']=df2['width']/W
    df2['height']=df2['height']/H
    
    df3 = df2.replace({'label': mapping})
    df4 = df3.drop(labels = 'bounding_box_data', axis = 1)
    
    df4.to_csv(r'/Users/fkarasek/Downloads/data_2/jpeg_labels/' + file_name + '.txt', header=None, index=None, sep=' ', mode='a')


In [ ]:
#Use the above function to loop through directory and print out txt files
from pathlib import Path


for filename in os.listdir('/Users/fkarasek/Downloads/data_2/labels_data'):
    mypath = '/Users/fkarasek/Downloads/data_2/labels_data/' + str(Path(filename))
    format_json(mypath)

In [ ]:
#find jpegs that have annotations and copy them to a separate folder
from shutil import copyfile


for filename in os.listdir('/Users/fkarasek/Downloads/data_2/jpeg_labels'):
    filenamext = (os.path.splitext(filename)[0])
    for file_name in os.listdir('/Users/fkarasek/Downloads/data_2/jpegs'):
        filenamextjp = (os.path.splitext(file_name)[0])
        if filenamext == filenamextjp:
            copyfile('/Users/fkarasek/Downloads/data_2/jpegs/'+ file_name, '/Users/fkarasek/Downloads/data_2/filtered_jpegs/'+file_name)
        else:
            continue

